# Tax-free childcare

[Official government guidance](https://www.gov.uk/tax-free-childcare?step-by-step-nav=d78aeaf6-1747-4d72-9619-f16efb4dd89d) | [Legislation](https://www.legislation.gov.uk/ukdsi/2015/9780111127063)

## Program description

### Overview

Tax-free childcare helps working families with childcare costs. For every **£8** paid into a childcare account, the government adds **£2**.

The program provides financial support up to **£2,000** per year (**£500** per quarter) for each eligible child. For disabled children, this support increases to **£4,000** per year (**£1,000** per quarter).

### Eligibility requirements

#### Age criteria
Standard childcare support extends until **September 1st** following the child's **11th** birthday. For **disabled children** who receive disability benefits or are certified as **blind/severely sight-impaired**, support continues until **September 1st** following their **16th** birthday.

#### Income requirements
Income thresholds apply at both ends of the spectrum. The maximum adjusted net income allowed is **£100,000** per year for each partner. Minimum earnings requirements (per **3 months**) vary by age:

| Age group | Minimum earnings |
|-----------|-----------------|
| **21+** | **£2,379** |
| **18-20** | **£1,788** |
| **Under 18/Apprentices** | **£1,331** |

#### Work status
Eligible individuals must be either **employed**, **self-employed**, or on qualifying leave (**sick**, **annual**, or **shared parental leave**). Applications can be submitted if starting work within **31 days**.

Non-working partners may still qualify if they receive any of these benefits: **incapacity benefit**, **severe disablement allowance**, **carer's allowance**, or **contribution-based employment and support allowance**.

### Additional rules

#### Income assessment
The program offers flexible income assessment:
* Combines **multiple income sources**
* Allows **annual income averaging**
* Gives **self-employed** individuals choice between total or self-employment income

Certain types of income are excluded from calculations: **dividends**, **interest**, **property income**, and **pension payments**.

#### Program interactions
Tax-free childcare is compatible with the **15/30 hours free childcare** program but cannot be combined with:
* **Working tax credit**
* **Child tax credit**
* **Universal credit**
* **Childcare vouchers** or directly contracted childcare

When approved for tax-free childcare:
* Existing benefits terminate automatically
* Childcare vouchers must be cancelled within **90 days**
* **New self-employed** businesses (under **12 months**) benefit from more flexible earnings requirements

## Implementation

[This section will describe how the program is implemented in PolicyEngine]